In [1]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install mysql-connector-python

  Using cached protobuf-4.21.12-cp37-abi3-macosx_10_9_universal2.whl (486 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.10.0 requires libclang>=13.0.0, which is not installed.
tensorflow 2.10.0 requires tensorflow-io-gcs-filesystem>=0.23.1, which is not installed.
tensorflow 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.21.12 which is incompatible.
tensorboard 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


Server running on port 8000
test123:N
test123:N


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 54147)
Traceback (most recent call last):
  File "/Users/johnny/anaconda3/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/johnny/anaconda3/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/johnny/anaconda3/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/johnny/anaconda3/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/johnny/anaconda3/lib/python3.10/http/server.py", line 433, in handle
    self.handle_one_request()
  File "/Users/johnny/anaconda3/lib/python3.10/http/server.py", line 421, in handle_one_request
    method()
  File "/var/folders/8s/9fh2s7w53w1fks_4dpp52tyw0000gn/T/ipykernel_88280/2

^C received, shutting down server


In [3]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("shibing624/text2vec-base-chinese")

sentence1_embeddings = model.encode(["go to the curch"])
# sentence2_embeddings = model.encode(["你是誰"])
# sentence2_embeddings = model.encode(["你過得如何"])
sentence2_embeddings = model.encode(["come on"])

similarity = util.pytorch_cos_sim(sentence1_embeddings, sentence2_embeddings)
print(similarity)

tensor([[0.6577]])


In [1]:
from http.server import BaseHTTPRequestHandler, HTTPServer
from sentence_transformers import SentenceTransformer, util
import mysql.connector
import json

def connect_to_database():
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="3659215fF",
        database="handle_together"
    )
    return connection

def send_json_response(self, status_code, data):
    # Convert status_code to integer
    status_code = int(status_code)

    # Set up the HTTP response
    self.send_response(status_code)
    self.send_header("Content-type", "application/json")
    self.end_headers()

    # Convert the data to JSON
    json_response = json.dumps(data)

    # Send the JSON response
    self.wfile.write(json_response.encode('utf-8'))

class WebServerHandler(BaseHTTPRequestHandler):

    def do_POST(self):       
        # Read the content length from the request headers
        content_length = int(self.headers['Content-Length'])

        # Read the JSON data from the request body
        post_data = self.rfile.read(content_length)
        json_data_list = json.loads(post_data.decode('utf-8'))

        if json_data_list:
            json_data = json_data_list[0]
            # Now json_data contains the parameters sent in the JSON format
            tag_array = json_data.get('tag_array', [])
            subject_array = json_data.get('subject_array', [])
            object_array = json_data.get('object_array', [])
            input_sentence = json_data.get('input_sentence', '')       
            public_ind = json_data.get('public_ind', '')
            command_ind = json_data.get('command_ind', '')

            print()
        # Load the Sentence Transformer model
        model = SentenceTransformer("shibing624/text2vec-base-chinese")

        # Fetch distinct subject codes from intent_type table
        connection = connect_to_database()
        cursor = connection.cursor()

        # Use parameterized query to avoid SQL injection
        # query = "SELECT tag, content FROM intent_training_data WHERE tag IN ({}) AND subject IN ({})".format(','.join(['%s' for _ in tag_array]), ','.join(['%s' for _ in subject_array]))
        if tag_array and any(tag_array):
            tag_condition = "tag IN ({})".format(','.join(['%s' for _ in tag_array]))
        else:
            tag_condition = "1=1"  # Always true if tag_array is empty or None
            
        if command_ind: 
            tag_condition2 = "AND COMMAND_IND = '"+command_ind+"'"
        else:
            tag_condition2 = ""  # Always true if tag_array is empty or None
        
        query = ("SELECT tag, content FROM intent_training_data WHERE {} AND subject IN ({}) AND object IN ({}) AND PUBLIC_IND = '"+public_ind+"'"+tag_condition2).format(tag_condition, ','.join(['%s' for _ in subject_array]) ,','.join(['%s' for _ in object_array]))
        params = tag_array + subject_array + object_array
        cursor.execute(query, params)
        rows = cursor.fetchall()
        candidate_responses_tags = [row[0] for row in rows]
        candidate_responses = [row[1] for row in rows]

        cursor.close()
        connection.close()

        # Encode input sentence and candidate responses
        input_embedding = model.encode([input_sentence])
        response_embeddings = model.encode(candidate_responses)

        # Calculate cosine similarity between input and responses
        similarities = util.pytorch_cos_sim(input_embedding, response_embeddings)[0]
        
        similarity = util.pytorch_cos_sim(input_embedding, response_embeddings)

        # Get the index of the most similar response
        most_similar_index = similarities.argmax().item()

        # Print the most similar response
        print("Input sentence:", input_sentence)
        print("Most tag:", candidate_responses_tags[most_similar_index])
        print("Most similar response:", candidate_responses[most_similar_index])
        print("similarity:",similarity)
        result = {
            "input_sentence": input_sentence,
            "tag": candidate_responses_tags[most_similar_index],
            "content": candidate_responses[most_similar_index],
            "similarity" : round(similarity.item(),5)
        }
        send_json_response(self, '200', result)

if __name__ == '__main__':
    try:
        port = 8000
        server = HTTPServer(('localhost', port), WebServerHandler)
        server.max_body_length = 10 * 1024 * 1024  # set to 10 MB

        print(f'Server running on port {port}')
        
        server.serve_forever()
    except KeyboardInterrupt:
        print('^C received, shutting down server')
        server.socket.close()


OSError: [Errno 48] Address already in use

In [ ]:
from http.server import BaseHTTPRequestHandler, HTTPServer
from sentence_transformers import SentenceTransformer, util
import mysql.connector
import json

def connect_to_database():
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="3659215fF",
        database="handle_together"
    )
    return connection

def send_json_response(self, status_code, data):
    # Convert status_code to integer
    status_code = int(status_code)

    # Set up the HTTP response
    self.send_response(status_code)
    self.send_header("Content-type", "application/json")
    self.end_headers()

    # Convert the data to JSON
    json_response = json.dumps(data)

    # Send the JSON response
    self.wfile.write(json_response.encode('utf-8'))

class WebServerHandler(BaseHTTPRequestHandler):

    def do_POST(self):       
        # Read the content length from the request headers
        content_length = int(self.headers['Content-Length'])

        # Read the JSON data from the request body
        post_data = self.rfile.read(content_length)
        json_data_list = json.loads(post_data.decode('utf-8'))

        if json_data_list:
            json_data = json_data_list[0]
            # Now json_data contains the parameters sent in the JSON format
            tag_array = json_data.get('tag_array', [])
            subject_array = json_data.get('subject_array', [])
            object_array = json_data.get('object_array', [])
            input_sentence = json_data.get('input_sentence', '')       
            public_ind = json_data.get('public_ind', '')
            command_ind = json_data.get('command_ind', '')

        # Load the Sentence Transformer model
        model = SentenceTransformer("shibing624/text2vec-base-chinese")

        # Fetch distinct subject codes from intent_type table
        connection = connect_to_database()
        cursor = connection.cursor()

        # Use parameterized query to avoid SQL injection
        # query = "SELECT tag, content FROM intent_training_data WHERE tag IN ({}) AND subject IN ({})".format(','.join(['%s' for _ in tag_array]), ','.join(['%s' for _ in subject_array]))
        if tag_array and any(tag_array):
            tag_condition = "tag IN ({})".format(','.join(['%s' for _ in tag_array]))
        else:
            tag_condition = "1=1"  # Always true if tag_array is empty or None
            
        if command_ind: 
            tag_condition2 = "AND COMMAND_IND = '"+command_ind+"'"
        else:
            tag_condition2 = "" 
        query = ("SELECT tag, content FROM intent_training_data WHERE {} AND subject IN ({}) AND object IN ({}) AND PUBLIC_IND = '"+public_ind+"'"+tag_condition2).format(tag_condition, ','.join(['%s' for _ in subject_array]) ,','.join(['%s' for _ in object_array]))
        params = tag_array + subject_array + object_array
        cursor.execute(query, params)
        rows = cursor.fetchall()
        candidate_responses_tags = [row[0] for row in rows]
        candidate_responses = [row[1] for row in rows]

        cursor.close()
        connection.close()

        # Encode input sentence and candidate responses
        input_embedding = model.encode([input_sentence])
        response_embedding = model.encode([candidate_responses])

        response_embeddings = model.encode(candidate_responses)

        # Calculate cosine similarity between input and responses
        similarities = util.pytorch_cos_sim(input_embedding, response_embeddings)[0]
        
        similarity = util.pytorch_cos_sim(input_embedding, response_embedding)

        # Get the index of the most similar response
        most_similar_index = similarities.argmax().item()

        # Print the most similar response
        print("Input sentence:", input_sentence)
        print("Most tag:", candidate_responses_tags[most_similar_index])
        print("Most similar response:", candidate_responses[most_similar_index])
        print("similarity:",similarity)
        result = {
            "input_sentence": input_sentence,
            "tag": candidate_responses_tags[most_similar_index],
            "content": candidate_responses[most_similar_index],
            "similarity" : round(similarity.item(),5)
        }
        send_json_response(self, '200', result)

if __name__ == '__main__':
    try:
        port = 8000
        server = HTTPServer(('localhost', port), WebServerHandler)
        server.max_body_length = 10 * 1024 * 1024  # set to 10 MB

        print(f'Server running on port {port}')
        
        server.serve_forever()
    except KeyboardInterrupt:
        print('^C received, shutting down server')
        server.socket.close()


Server running on port 8000
Input sentence: 大家好
Most tag: CALL
Most similar response: 各位大家好
similarity: tensor([[0.8370]])


127.0.0.1 - - [18/Dec/2023 10:37:24] "POST / HTTP/1.1" 200 -


Input sentence: 大家好
Most tag: CALL_BACK
Most similar response: 版主好啊
similarity: tensor([[0.4415]])


127.0.0.1 - - [18/Dec/2023 10:37:26] "POST / HTTP/1.1" 200 -


Input sentence: 各位想看什麼呀
Most tag: ASK_HOW_TO_PLAY
Most similar response: 各位想要看什麼呢
similarity: tensor([[0.4350]])


...

127.0.0.1 - - [18/Dec/2023 10:47:49] "POST / HTTP/1.1" 200 -


Input sentence: 各位想看什麼表演呀
Most tag: ASK_HOW_TO_PLAY
Most similar response: 各位想要看什麼呢
similarity: tensor([[0.4709]])


127.0.0.1 - - [18/Dec/2023 10:51:35] "POST / HTTP/1.1" 200 -


Input sentence: 各位想看什麼表演呀
Most tag: ASK_HOW_TO_PLAY
Most similar response: 各位想要看什麼呢
similarity: tensor([[0.4709]])


127.0.0.1 - - [18/Dec/2023 11:02:22] "POST / HTTP/1.1" 200 -
